<a href="https://colab.research.google.com/github/manthanawgan/MLops-Projects/blob/main/Nagpur_Temperature_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression

* Temp unit :- celsius
* wind speed unit :- kn/h
* precipitation unit :- millimeter


In [2]:
df = pd.read_csv('/content/historical-weather-dataset-ngp.csv')

df.head()

,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),sunshine_duration (s),shortwave_radiation_sum (MJ/m²),weather_code (wmo code),apparent_temperature_mean (°C),relative_humidity_2m_mean (%),dew_point_2m_mean (°C),cloud_cover_mean (%),wind_speed_10m_mean (km/h),surface_pressure_mean (hPa),et0_fao_evapotranspiration_sum (mm),soil_temperature_0_to_7cm_mean (°C),soil_moisture_0_to_10cm_mean (undefined)
0,1/1/2013,24.7,30.5,20.8,35383.75,13.64,61,27.8,78.0,20.3,26.0,6.2,975.5,2.98,25.8,NaN
1,1/2/2013,24.1,29.4,18.1,35423.55,13.53,2,26.7,74.0,19.0,18.0,6.2,976.3,3.03,25.4,NaN
2,1/3/2013,21.3,27.6,15.6,35615.22,17.20,0,20.9,57.0,11.9,0.0,7.8,978.1,3.79,23.6,NaN
3,1/4/2013,20.8,27.3,14.7,35729.32,17.47,0,20.6,56.0,11.4,0.0,6.0,976.8,3.62,23.0,NaN
4,1/5/2013,21.5,27.7,15.5,35696.16,15.66,0,21.4,57.0,12.3,6.0,6.8,974.7,3.52,23.3,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4511 entries, 0 to 4510
Data columns (total 16 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   time                                      4511 non-null   object 
 1   temperature_2m_mean (°C)                  4510 non-null   float64
 2   temperature_2m_max (°C)                   4511 non-null   float64
 3   temperature_2m_min (°C)                   4511 non-null   float64
 4   sunshine_duration (s)                     4510 non-null   float64
 5   shortwave_radiation_sum (MJ/m²)           4510 non-null   float64
 6   weather_code (wmo code)                   4511 non-null   int64  
 7   apparent_temperature_mean (°C)            4510 non-null   float64
 8   relative_humidity_2m_mean (%)             4510 non-null   float64
 9   dew_point_2m_mean (°C)                    4510 non-null   float64
 10  cloud_cover_mean (%)                

In [4]:
df.describe()

,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),sunshine_duration (s),shortwave_radiation_sum (MJ/m²),weather_code (wmo code),apparent_temperature_mean (°C),relative_humidity_2m_mean (%),dew_point_2m_mean (°C),cloud_cover_mean (%),wind_speed_10m_mean (km/h),surface_pressure_mean (hPa),et0_fao_evapotranspiration_sum (mm),soil_temperature_0_to_7cm_mean (°C),soil_moisture_0_to_10cm_mean (undefined)
count,4510.000000,4511.000000,4511.000000,4510.000000,4510.000000,4511.000000,4510.000000,4510.000000,4510.000000,4510.000000,4510.000000,4510.000000,4510.000000,4510.000000,0.0
mean,26.787206,32.388051,21.539570,34350.735945,18.465769,24.374640,28.222838,57.821286,15.906408,44.277827,8.488736,973.050421,4.752191,28.487273,NaN
std,4.698131,5.001981,5.001892,9947.990816,4.811463,27.553258,5.396295,21.433967,6.211279,36.154688,3.045820,5.040345,1.896309,5.112904,NaN
min,14.400000,18.500000,7.600000,0.000000,1.370000,0.000000,11.800000,12.000000,-3.000000,0.000000,2.600000,958.800000,0.740000,17.000000,NaN
25%,23.600000,29.000000,17.500000,35200.187500,16.160000,2.000000,24.000000,41.000000,10.900000,12.000000,6.300000,968.800000,3.550000,25.000000,NaN
50%,26.400000,31.000000,22.900000,37022.365000,18.750000,3.000000,29.500000,58.000000,15.500000,35.000000,7.700000,973.200000,4.280000,27.600000,NaN
75%,29.000000,35.300000,24.700000,39857.645000,21.840000,53.000000,32.200000,77.000000,22.400000,82.000000,9.900000,977.500000,5.910000,31.100000,NaN
max,39.300000,45.800000,34.300000,44484.390000,28.090000,65.000000,39.400000,95.000000,25.900000,100.000000,29.200000,983.400000,12.340000,42.200000,NaN


In [5]:
df.isnull().sum()

,0
time,0
temperature_2m_mean (°C),1
temperature_2m_max (°C),0
temperature_2m_min (°C),0
sunshine_duration (s),1
shortwave_radiation_sum (MJ/m²),1
weather_code (wmo code),0
apparent_temperature_mean (°C),1
relative_humidity_2m_mean (%),1
dew_point_2m_mean (°C),1


In [8]:
df.rename(columns={
    'temperature_2m_mean (°C)': 'temp_mean',
    'temperature_2m_max (°C)': 'temp_max',
    'temperature_2m_min (°C)': 'temp_min',
    'sunshine_duration (s)': 'sunshine_duration',
    'shortwave_radiation_sum (MJ/m²)': 'radiation_sum',
    'weather_code (wmo code)': 'weather_code',
    'apparent_temperature_mean (°C)': 'apparent_mean_temp',
    'relative_humidity_2m_mean (%)': 'relative_humidity',
    'dew_point_2m_mean (°C)': 'dew_point_mean',
    'cloud_cover_mean (%)': 'cloud_cover_mean',
    'wind_speed_10m_mean (km/h)': 'wind_speed_mean',
    'surface_pressure_mean (hPa)': 'pressure_mean',
    'et0_fao_evapotranspiration_sum (mm)': 'evapotranspiration_sum',
    'soil_temperature_0_to_7cm_mean (°C)': 'soil_temp_mean',
    'soil_moisture_0_to_10cm_mean (undefined)': 'soil_moisture_mean'
})

,time,temp_mean,temp_max,temp_min,sunshine_duration,radiation_sum,weather_code,apparent_mean_temp,relative_humidity,dew_point_mean,cloud_cover_mean,wind_speed_mean,pressure_mean,evapotranspiration_sum,soil_temp_mean,soil_moisture_mean
0,1/1/2013,24.7,30.5,20.8,35383.75,13.64,61,27.8,78.0,20.3,26.0,6.2,975.5,2.98,25.8,NaN
1,1/2/2013,24.1,29.4,18.1,35423.55,13.53,2,26.7,74.0,19.0,18.0,6.2,976.3,3.03,25.4,NaN
2,1/3/2013,21.3,27.6,15.6,35615.22,17.20,0,20.9,57.0,11.9,0.0,7.8,978.1,3.79,23.6,NaN
3,1/4/2013,20.8,27.3,14.7,35729.32,17.47,0,20.6,56.0,11.4,0.0,6.0,976.8,3.62,23.0,NaN
4,1/5/2013,21.5,27.7,15.5,35696.16,15.66,0,21.4,57.0,12.3,6.0,6.8,974.7,3.52,23.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4506,5/4/2025,31.4,38.6,25.3,40188.08,24.11,51,32.4,39.0,15.0,15.0,7.4,970.7,6.68,36.1,NaN
4507,5/5/2025,32.6,39.3,25.7,41049.59,24.09,3,32.8,33.0,13.0,25.0,8.1,970.7,7.28,37.0,NaN
4508,5/6/2025,31.9,38.1,25.4,42207.39,25.30,3,31.9,34.0,12.8,17.0,9.3,971.4,7.63,36.4,NaN
4509,5/7/2025,31.2,36.9,25.8,42253.05,25.09,2,31.4,38.0,14.1,12.0,10.4,970.7,7.61,35.7,NaN
